<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Websites" data-toc-modified-id="Websites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Websites</a></span></li><li><span><a href="#TV-models" data-toc-modified-id="TV-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>TV models</a></span></li><li><span><a href="#TV-brands" data-toc-modified-id="TV-brands-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TV brands</a></span></li></ul></div>

# Analysis on penetration pricing

In [1]:
# Loading prerequisite libraries

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go
import plotly.express as px

import os
if not os.path.exists("images"):
    os.mkdir("images")

In [2]:
# Load CSV to a pandas dataframe

df = pd.read_csv('Products.csv',infer_datetime_format=True, parse_dates=True)

In [4]:
# filter only useful columns

df = df[["Price-Max", "Price-Min", "Availability", "Date-Seen", "On-Sale", "Merchant", "Brand", "Date-Added", "Date-Updated", "Name", "Manufacturer-Id", "Price-Source-URLs"]]

In [5]:
# create a time series

df = df.drop('Date-Seen', axis=1).join(
  df['Date-Seen'].str.split(',', expand=True).stack().reset_index(level=1,drop=True).rename('Date-Seen'))

df['Date-Seen'] = pd.to_datetime(df['Date-Seen'])

In [6]:
df.head(2)

,Price-Max,Price-Min,Availability,On-Sale,Merchant,Brand,Date-Added,Date-Updated,Name,Manufacturer-Id,Price-Source-URLs,Date-Seen
0,799.99,799.99,In Stock,False,Bestbuy.com,Hisense,2018-03-16T01:47:45Z,2018-06-13T19:36:51Z,Hisense - 55 class - led - h9 series - 2160p -...,55H9D PLUS,https://www.bestbuy.com/site/hisense-55-class-...,2018-05-26 15:00:00+00:00
0,799.99,799.99,In Stock,False,Bestbuy.com,Hisense,2018-03-16T01:47:45Z,2018-06-13T19:36:51Z,Hisense - 55 class - led - h9 series - 2160p -...,55H9D PLUS,https://www.bestbuy.com/site/hisense-55-class-...,2018-05-02 14:00:00+00:00


## Websites

In [7]:
# how many websites do we have?

df["Website"] = df["Price-Source-URLs"].str.extract(r'\.([^\.]+)\.')
list(df["Website"].unique())

['bestbuy', 'walmart', 'ebay', 'bhphotovideo']

## TV models

In [8]:
list(df["Manufacturer-Id"].unique())

['55H9D PLUS',
 'UN55KU7000FXZA',
 'XBR55A1E',
 'QN55Q8FNBFXZA',
 'XBR55X900F',
 'UN55MU6490FXZA',
 '55UJ7700',
 'QN55Q7CAMFXZA',
 '55H8C',
 'XBR55X700D',
 'XBR55X800E']

## TV brands

In [9]:
list(df["Brand"].unique())

['Hisense', 'Samsung', 'Sony', 'Lg']

# 55-inch TV prices on each website

In [10]:
# # Create as many colors as there are unique df['Website']
# Website = np.unique(df['Website'])
# colors = ["red", "blue", "green", "black"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, Website in enumerate(Website):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df.loc[df.Website==Website, :], 
#                 s=50, c=colors[i], label=str(Website))
    
# plt.legend(fontsize=12)

In [11]:
# figure, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,10))
# ax1.scatter('Date-Seen', 'Price-Min', data=df.loc[(df.Website=="bestbuy") & (df["On-Sale"]==True)], s=50, c="red")
# ax1.scatter('Date-Seen', 'Price-Min', data=df.loc[(df.Website=="bestbuy") & (df["On-Sale"]==False)], s=50, c="blue")
# ax2.scatter('Date-Seen', 'Price-Min', data=df.loc[(df.Website=="bhphotovideo")  & (df["On-Sale"]==True)], s=50, c="red")
# ax2.scatter('Date-Seen', 'Price-Min', data=df.loc[(df.Website=="bhphotovideo")  & (df["On-Sale"]==False)], s=50, c="blue")
# ax3.scatter('Date-Seen', 'Price-Min', data=df.loc[(df.Website=="ebay")  & (df["On-Sale"]==True)], s=50, c="red")
# ax3.scatter('Date-Seen', 'Price-Min', data=df.loc[(df.Website=="ebay")  & (df["On-Sale"]==False)], s=50, c="blue")
# ax4.scatter('Date-Seen', 'Price-Min', data=df.loc[(df.Website=="walmart")  & (df["On-Sale"]==True)], s=50, c="red")
# ax4.scatter('Date-Seen', 'Price-Min', data=df.loc[(df.Website=="walmart")  & (df["On-Sale"]==False)], s=50, c="blue")
# ax1.title.set_text('Besbuy')
# ax2.title.set_text('BHPhotovideo')
# ax3.title.set_text('eBay')
# ax4.title.set_text('Walmart')

In [39]:
fig = px.scatter(df, x="Date-Seen", y="Price-Min",color="On-Sale", facet_row="Website", height=800)

#styling
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(tickprefix="$")
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='')
fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers

fig.show()

In [40]:
# fig.write_image("images/sale_prices_by_website.png")

In [14]:
# plot with plotly
fig1 = px.scatter(df.loc[df["Website"] == "bestbuy"], x="Date-Seen", y="Price-Min",color="On-Sale")

# styling
fig1.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig1.update_xaxes(title_text='') # get rid of x-title
fig1.update_yaxes(title_text='') # get rid of y-title
fig1.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig1.update_layout(
    title="Bestbuy",
    legend_title="On-Sale",
    font=dict(
        size=12,
    )
)

fig1.show()

####################################################################################

# plot with plotly
fig2 = px.scatter(df.loc[df["Website"] == "walmart"], x="Date-Seen", y="Price-Min",color="On-Sale")

# styling
fig2.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig2.update_xaxes(title_text='') # get rid of x-title
fig2.update_yaxes(title_text='') # get rid of y-title
fig2.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig2.update_layout(
    title="Walmart",
    legend_title="On-Sale",
    font=dict(
        size=12,
    )
)

fig2.show()

################################################################################################

# plot with plotly
fig3 = px.scatter(df.loc[df["Website"] == "ebay"], x="Date-Seen", y="Price-Min",color="On-Sale")

# styling
fig3.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig3.update_xaxes(title_text='') # get rid of x-title
fig3.update_yaxes(title_text='') # get rid of y-title
fig3.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig3.update_layout(
    title="eBay",
    legend_title="On-Sale",
    font=dict(
        size=12,
    )
)

fig3.show()

################################################################################################

# plot with plotly
fig4 = px.scatter(df.loc[df["Website"] == "bhphotovideo"], x="Date-Seen", y="Price-Min",color="On-Sale")

# styling
fig4.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig4.update_xaxes(title_text='') # get rid of x-title
fig4.update_yaxes(title_text='') # get rid of y-title
fig4.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig4.update_layout(
    title="B&H Photo Video",
    legend_title="On-Sale",
    font=dict(
        size=12,
    )
)

fig4.show()

# Evolution of prices for each model in each website

In [15]:
# plot with plotly
fig1 = px.scatter(df.loc[df["Website"] == "bestbuy"], x="Date-Seen", y="Price-Min",color="Manufacturer-Id")

# styling
fig1.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig1.update_xaxes(title_text='') # get rid of x-title
fig1.update_yaxes(title_text='') # get rid of y-title
fig1.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig1.update_layout(showlegend=False,
#     title="Bestbuy",
    legend_title="Models",
    font=dict(
        size=12,
    )
)

fig1.show()

################################################################################################


# plot with plotly
fig2 = px.scatter(df.loc[df["Website"] == "walmart"], x="Date-Seen", y="Price-Min",color="Manufacturer-Id")

# styling
fig2.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig2.update_xaxes(title_text='') # get rid of x-title
fig2.update_yaxes(title_text='') # get rid of y-title
fig2.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig2.update_layout(showlegend=False,
#     title="Walmart",
    legend_title="Models",
    font=dict(
        size=12,
    )
)

fig2.show()

################################################################################################


# plot with plotly
fig3 = px.scatter(df.loc[df["Website"] == "ebay"], x="Date-Seen", y="Price-Min",color="Manufacturer-Id")

# styling
fig3.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig3.update_xaxes(title_text='') # get rid of x-title
fig3.update_yaxes(title_text='') # get rid of y-title
fig3.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig3.update_layout(showlegend=False,
#     title="eBay",
    legend_title="Models",
    font=dict(
        size=12,
    )
)

fig3.show()

################################################################################################

# plot with plotly
fig4 = px.scatter(df.loc[df["Website"] == "bhphotovideo"], x="Date-Seen", y="Price-Min",color="Manufacturer-Id")

# styling
fig4.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig4.update_xaxes(title_text='') # get rid of x-title
fig4.update_yaxes(title_text='') # get rid of y-title
fig4.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig4.update_layout(showlegend=False,
#     title="B&H Photo Video",
    legend_title="Models",
    font=dict(
        size=12,
    )
)

fig4.show()

In [16]:
# fig1.write_image("images/bestbuy_time_series.png")
# fig2.write_image("images/walmart_time_series.png")
# fig3.write_image("images/ebay_time_series.png")
# fig4.write_image("images/bhpv_time_series.png")

In [17]:
fig = px.scatter(df, x="Date-Seen", y="Price-Min",color="Manufacturer-Id", facet_row="Website", height=800)

#styling
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(tickprefix="$")
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='')
fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig.update_layout(showlegend=False)
fig.show()

In [18]:
# fig.write_image("images/products_by_website_facet.png")

In [19]:
# df1 = df[df["Website"] == "bestbuy"]

# # Create as many colors as there are unique df['Website']
# ManufacturerId = np.unique(df1['Manufacturer-Id'])
# colors = ["red", "blue", "green", "black", "pink", "slategray", "magenta", "cyan", "gold", "lime"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, ManufacturerId in enumerate(ManufacturerId):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df1.loc[df1["Manufacturer-Id"] == ManufacturerId, :], 
#                 s=50, c=colors[i], label=str(ManufacturerId))
    
# plt.title("Bestbuy", fontsize=22)
# plt.legend(fontsize=12)

In [20]:
# df1 = df[df["Website"] == "ebay"]

# # Create as many colors as there are unique df['Website']
# ManufacturerId = np.unique(df1['Manufacturer-Id'])
# colors = ["red", "blue", "green", "black", "pink", "slategray", "magenta", "cyan", "gold", "lime"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, ManufacturerId in enumerate(ManufacturerId):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df1.loc[df1["Manufacturer-Id"] == ManufacturerId, :], 
#                 s=50, c=colors[i], label=str(ManufacturerId))
    
# plt.title("eBay", fontsize=22)
# plt.legend(fontsize=12)

In [21]:
# df1 = df[df["Website"] == "walmart"]

# # Create as many colors as there are unique df['Website']
# ManufacturerId = np.unique(df1['Manufacturer-Id'])
# colors = ["red", "blue", "green", "black", "pink", "slategray", "magenta", "cyan", "gold", "lime", "indigo"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, ManufacturerId in enumerate(ManufacturerId):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df1.loc[df1["Manufacturer-Id"] == ManufacturerId, :], 
#                 s=50, c=colors[i], label=str(ManufacturerId))
    
# plt.title("Walmart", fontsize=22)
# plt.legend(fontsize=12)

In [22]:
# df1 = df[df["Website"] == "bhphotovideo"]

# # Create as many colors as there are unique df['Website']
# ManufacturerId = np.unique(df1['Manufacturer-Id'])
# colors = ["red", "blue", "green", "black", "pink", "slategray", "magenta", "cyan", "gold", "lime"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, ManufacturerId in enumerate(ManufacturerId):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df1.loc[df1["Manufacturer-Id"] == ManufacturerId, :], 
#                 s=50, c=colors[i], label=str(ManufacturerId))
    
# plt.title("BHPhotovideo", fontsize=22)
# plt.legend(fontsize=12)

# Prices overtime for each brand

In [23]:
# # Create as many colors as there are unique df['Website']
# Brand = np.unique(df['Brand'])
# colors = ["red", "blue", "green", "black"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, Brand in enumerate(Brand):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df.loc[df.Brand==Brand, :], 
#                 s=50, c=colors[i], label=str(Brand))

# plt.title("Prices across all websites for each brand", fontsize=22)
# plt.legend(fontsize=12)

In [24]:
# plot with plotly
fig = px.scatter(df.loc[df["Brand"] == "Hisense"], x="Date-Seen", y="Price-Min",color="Manufacturer-Id")

# styling
fig.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig.update_xaxes(title_text='') # get rid of x-title
fig.update_yaxes(title_text='') # get rid of y-title
fig.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig.update_layout(
    title="Hisense prices across all platforms",
    legend_title="Models",
    font=dict(
        size=12,
    )
)

fig.show()

In [25]:
# df1 = df[df["Brand"] == "Hisense"]

# # Create as many colors as there are unique df['Website']
# ManufacturerId = np.unique(df1['Manufacturer-Id'])
# colors = ["red", "blue", "green", "black", "pink", "slategray", "magenta", "cyan", "gold", "lime"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, ManufacturerId in enumerate(ManufacturerId):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df1.loc[df1["Manufacturer-Id"] == ManufacturerId, :], 
#                 s=50, c=colors[i], label=str(ManufacturerId))
    
# plt.title("Hisense prices across all websites", fontsize=22)
# plt.legend(fontsize=12)

In [26]:
# figure, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,10))
# ax1.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="bestbuy"], s=50, c="red")
# ax2.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="bhphotovideo"], s=50, c="blue")
# ax3.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="ebay"], s=50, c="green")
# ax4.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="walmart"], s=50, c="black")
# ax1.title.set_text('Hisense - Besbuy')
# ax2.title.set_text('Hisense - BHPhotovideo')
# ax3.title.set_text('Hisense - eBay')
# ax4.title.set_text('Hisense - Walmart')

In [27]:
# plot with plotly
fig = px.scatter(df.loc[df["Brand"] == "Lg"], x="Date-Seen", y="Price-Min",color="Manufacturer-Id")

# styling
fig.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig.update_xaxes(title_text='') # get rid of x-title
fig.update_yaxes(title_text='') # get rid of y-title
fig.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig.update_layout(
    title="LG prices across all platforms",
    legend_title="Models",
    font=dict(
        size=12,
    )
)

fig.show()

In [28]:
# df1 = df[df["Brand"] == "Lg"]

# # Create as many colors as there are unique df['Website']
# ManufacturerId = np.unique(df1['Manufacturer-Id'])
# colors = ["red", "blue", "green", "black", "pink", "slategray", "magenta", "cyan", "gold", "lime"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, ManufacturerId in enumerate(ManufacturerId):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df1.loc[df1["Manufacturer-Id"] == ManufacturerId, :], 
#                 s=50, c=colors[i], label=str(ManufacturerId))
    
# plt.title("LG prices across all websites", fontsize=22)
# plt.legend(fontsize=12)

In [29]:
# figure, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,10))
# ax1.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="bestbuy"], s=50, c="red")
# ax2.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="bhphotovideo"], s=50, c="blue")
# ax3.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="ebay"], s=50, c="green")
# ax4.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="walmart"], s=50, c="black")
# ax1.title.set_text('LG - Besbuy')
# ax2.title.set_text('LG - BHPhotovideo')
# ax3.title.set_text('LG - eBay')
# ax4.title.set_text('LG - Walmart')

In [30]:
# plot with plotly
fig = px.scatter(df.loc[df["Brand"] == "Samsung"], x="Date-Seen", y="Price-Min",color="Manufacturer-Id")

# styling
fig.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig.update_xaxes(title_text='') # get rid of x-title
fig.update_yaxes(title_text='') # get rid of y-title
fig.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig.update_layout(
    title="Samsung prices across all platforms",
    legend_title="Models",
    font=dict(
        size=12,
    )
)

fig.show()

In [31]:
# df1 = df[df["Brand"] == "Samsung"]

# # Create as many colors as there are unique df['Website']
# ManufacturerId = np.unique(df1['Manufacturer-Id'])
# colors = ["red", "blue", "green", "black", "pink", "slategray", "magenta", "cyan", "gold", "lime"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, ManufacturerId in enumerate(ManufacturerId):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df1.loc[df1["Manufacturer-Id"] == ManufacturerId, :], 
#                 s=50, c=colors[i], label=str(ManufacturerId))
    
# plt.title("Samsung prices across all websites", fontsize=22)
# plt.legend(fontsize=12)

In [32]:
# figure, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,10))
# ax1.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="bestbuy"], s=50, c="red")
# ax2.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="bhphotovideo"], s=50, c="blue")
# ax3.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="ebay"], s=50, c="green")
# ax4.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="walmart"], s=50, c="black")
# ax1.title.set_text('Samsung - Besbuy')
# ax2.title.set_text('Samsung - BHPhotovideo')
# ax3.title.set_text('Samsung - eBay')
# ax4.title.set_text('Samsung - Walmart')

In [33]:
# plot with plotly
fig = px.scatter(df.loc[df["Brand"] == "Sony"], x="Date-Seen", y="Price-Min",color="Manufacturer-Id")

# styling
fig.update_yaxes(tickprefix="$") # dollar sign on y-axis
fig.update_xaxes(title_text='') # get rid of x-title
fig.update_yaxes(title_text='') # get rid of y-title
fig.update_traces(marker=dict(size=7, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers')) # style markers
fig.update_layout(
    title="Sony prices across all platforms",
    legend_title="Models",
    font=dict(
        size=12,
    )
)

fig.show()

In [34]:
# df1 = df[df["Brand"] == "Sony"]

# # Create as many colors as there are unique df['Website']
# ManufacturerId = np.unique(df1['Manufacturer-Id'])
# colors = ["red", "blue", "green", "black", "pink", "slategray", "magenta", "cyan", "gold", "lime"]


# # Draw Plot for Each Website
# plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')

# for i, ManufacturerId in enumerate(ManufacturerId):
#     plt.scatter('Date-Seen', 'Price-Min', 
#                 data=df1.loc[df1["Manufacturer-Id"] == ManufacturerId, :], 
#                 s=50, c=colors[i], label=str(ManufacturerId))
    
# plt.title("Sony prices across all websites", fontsize=22)
# plt.legend(fontsize=12)

In [35]:
# figure, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,10))
# ax1.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="bestbuy"], s=50, c="red")
# ax2.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="bhphotovideo"], s=50, c="blue")
# ax3.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="ebay"], s=50, c="green")
# ax4.scatter('Date-Seen', 'Price-Min', data=df1.loc[df1.Website=="walmart"], s=50, c="black")
# ax1.title.set_text('Sony - Besbuy')
# ax2.title.set_text('Sony - BHPhotovideo')
# ax3.title.set_text('Sony - eBay')
# ax4.title.set_text('Sony - Walmart')

In [36]:
dfAvg = df.groupby("Brand").mean()
dfAvg.sort_values("Price-Min")

,Price-Max,Price-Min,On-Sale
Brand,,,
Hisense,595.344667,583.764000,0.200000
Lg,1194.392000,879.234000,0.700000
Samsung,2120.806225,1693.442318,0.549669
Sony,1789.733094,1716.444126,0.246637


# On sale

In [37]:
dfAvg = df.groupby(["Brand", "On-Sale"]).mean()
dfAvg

Price-Max    Price-Min
Brand   On-Sale                          
Hisense False     584.916250   584.916250
        True      637.058333   579.155000
Lg      False    1131.323333  1131.323333
        True     1221.421429   771.195714
Samsung False    1650.552353  1650.552353
        True     2506.074458  1728.581084
Sony    False    1711.581429  1711.581429
        True     2028.450909  1731.297455